In [45]:
# from https://code.datasciencedojo.com/datasciencedojo/datasets/tree/master/Blood%20Transfusion%20Service%20Center

import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer

import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set a random seed
# np.random.seed(42)

### Data load

In [46]:
data_df = pd.read_csv('./transfusion.data.csv')
data_df.rename(columns={'Monetary (c.c. blood)': 'Volume (c.c. blood)'}, inplace=True)

short_df = data_df.copy()
for col in short_df.columns:
    shortname = col.split()[0]
    short_df.rename(columns={col:shortname}, inplace=True)
    
X = short_df[short_df.columns[:-1]]
y = short_df['whether']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.array(y_train, requires_grad = False)
y_test = np.array(y_test, requires_grad = False)

### Useful functions

In [47]:
def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data

    Args:
        inputs (array[float]): input data
        targets (array[float]): targets

    Returns:
        inputs (array[float]): one batch of input data of length `batch_size`
        targets (array[float]): one batch of targets of length `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [48]:
# Define output labels as quantum state vectors
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T

### Quantum circuits

In [49]:
def U_SU4(params, wires):  # General circuit for 2 qubits, 15 params
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])

def Pooling_ansatz1(params, wires): #2 params
    qml.CRZ(params[0], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

In [50]:
dev = qml.device("lightning.qubit", wires=4)
@qml.qnode(dev, interface="autograd", diff_method ="adjoint")
def QCNN(params, x, M):
    p1 = params[0:15]
    p2 = params[15:30]
    p3 = params[30:32]
    p4 = params[32:34]
    p5 = params[34:49]
    p6 = params[49:51]
    
    qml.RY(x[0], wires=0)
    qml.RY(x[1], wires=1)
    qml.RY(x[2], wires=2)
    qml.RY(x[3], wires=3)
    
    U_SU4(p1, [0,1])
    U_SU4(p2, [2,3])
    Pooling_ansatz1(p3, [1,0])
    Pooling_ansatz1(p4, [3,2])
    U_SU4(p5, [0,2])
    Pooling_ansatz1(p6, [0,2])
    
    return qml.expval(qml.Hermitian(M, wires=[2]))

dev = qml.device("lightning.qubit", wires=4)
@qml.qnode(dev, interface="autograd", diff_method="adjoint")
def QCNN_reuploading(params, x, M):

    p1 = params[0:15]
    p2 = params[15:30]
    p3 = params[30:32]
    p4 = params[32:34]
    p5 = params[34:49]
    p6 = params[49:51]
    
    qml.RY(x[0], wires=0)
    qml.RY(x[1], wires=1)
    qml.RY(x[2], wires=2)
    qml.RY(x[3], wires=3)
    
    U_SU4(p1, [0,1])
    U_SU4(p2, [2,3])
    Pooling_ansatz1(p3, [1,0])
    Pooling_ansatz1(p4, [3,2])
    
    qml.Rot(x[0], x[1], 0, wires = 0)
    qml.Rot(x[2], x[3], 0, wires = 2)
    
    U_SU4(p5, [0,2])
    Pooling_ansatz1(p6, [0,2])

    return qml.expval(qml.Hermitian(M, wires=[0]))

### Cost

In [51]:
def cost(params, x, y, state_labels=None):
    """Cost function to be minimized.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): array of state representations for labels

    Returns:
        float: loss value to be minimized
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    # print(dm_labels) #!------------------------------------------------------------------------ Mirar si surt 3D: no, surt bé --------------------------
    for i in range(len(x)):
        f = QCNN(params, x[i], dm_labels[y[i]])
        loss = loss + (1 - f) ** 2  #!-------------------------------------------------------- usem cross entropy? ------------------------
    return loss / len(x)

### Test and accuracy

In [52]:

def test(params, x, y, state_labels=None):
    """
    Tests on a given set of data.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        predicted (array([int]): predicted labels for test data
        output_states (array[float]): output quantum states from the circuit
    """
    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        fidel_function = lambda y: qcircuit(params, x[i], y)
        fidelities = [fidel_function(dm) for dm in dm_labels]
        best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(predicted), np.array(fidelity_values)


def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    score = y_true == y_pred
    return score.sum() / len(y_true)

### Execution

In [59]:
# Train using Adam optimizer and evaluate the classifier
qcircuit = QCNN_reuploading # QCNN or QCNN_reuploading
learning_rate = 0.05
epochs = 10
batch_size = 30

label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = np.array([label_0, label_1], requires_grad=False)

# print(state_labels)
opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

# initialize random weights
if qcircuit == QCNN:
    params = np.random.uniform(size=(51), requires_grad=True)
    circuit_name = "QCNN"

elif qcircuit == QCNN_reuploading:
    params = np.random.uniform(size=(51), requires_grad=True)
    circuit_name = "QCNN_reuploading"

cost_history = []
for it in range(epochs):
    for Xbatch, ybatch in iterate_minibatches(X_train, y_train, batch_size=batch_size):
        variables, cost_value = opt.step_and_cost(cost, params, Xbatch, ybatch, state_labels)
        params = variables[0]
        
    print(f'Epoch: {it + 1} | Cost: {cost_value}')
    cost_history.append(float(cost_value))
    
predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
accuracy_test = accuracy_score(y_test, predicted_test)

print(f"{circuit_name}, epochs = {epochs}, acc = {accuracy_test}")

with open('Results/results.txt', 'a') as f:
    f.write(datetime.now().strftime("%d/%m/%Y %H:%M:%S") + "\n")
    f.write(f"Cost History for {circuit_name}, epochs = {epochs}, acc = {accuracy_test}\n")
    f.write(str(cost_history) + "\n\n")


Epoch: 1 | Cost: 0.2051568498385202
Epoch: 2 | Cost: 0.1962074556397591
Epoch: 3 | Cost: 0.19357864335849825
Epoch: 4 | Cost: 0.18912507918727187
Epoch: 5 | Cost: 0.18693347190479565
Epoch: 6 | Cost: 0.18524279222901732
Epoch: 7 | Cost: 0.18441560562427062
Epoch: 8 | Cost: 0.18404535801647087
Epoch: 9 | Cost: 0.18420115880258855
Epoch: 10 | Cost: 0.1839822298714257
QCNN_reuploading, epochs = 10, acc = 0.7540106951871658
